In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import time

driver = webdriver.Chrome()

# our data uses patches 13_1, 13_2, 13_4 and 13_5
url = 'https://u.gg/lol/tier-list?patch=13_4&rank=diamond'
driver.get(url)

# scroll the page to load all the champion names
while len(driver.find_elements(By.CSS_SELECTOR, 'strong.champion-name')) < 226: # change value based on total champion names per page
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(1)

# extract the champion names, win rates, pick rates, and ban rates
champion_names = []
win_rates = []
pick_rates = []
ban_rates = []
for i, row in enumerate(driver.find_elements(By.CSS_SELECTOR, 'div.rt-tr-group')):
    # find the champion name and win rates 
    champion_name = row.find_element(By.CSS_SELECTOR, 'strong.champion-name').text.strip()
    win_rate_class = 'div.rt-td.winrate.sorted.is-in-odd-row' if i % 2 == 0 else 'div.rt-td.winrate.sorted'
    win_rate = row.find_element(By.CSS_SELECTOR, win_rate_class).text.strip()
    win_rate = float(win_rate[:-1]) / 100  # convert win rate to decimal
    
    # find the pick rate and ban rate 
    pick_rate_class = 'div.rt-td.pickrate.is-in-odd-row' if i % 2 == 0 else 'div.rt-td.pickrate'
    pick_rate = row.find_element(By.CSS_SELECTOR, pick_rate_class).text.strip()
    pick_rate = float(pick_rate[:-1]) / 100  # convert pick rate to decimal
    ban_rate_class = 'div.rt-td.banrate.is-in-odd-row' if i % 2 == 0 else 'div.rt-td.banrate'
    ban_rate = row.find_element(By.CSS_SELECTOR, ban_rate_class).text.strip()
    ban_rate = float(ban_rate[:-1]) / 100  # convert ban rate to decimal
    
    champion_names.append(champion_name)
    win_rates.append(win_rate)
    pick_rates.append(pick_rate)
    ban_rates.append(ban_rate)

data = {'championName': champion_names, 'win_rate': win_rates, 'pick_rate': pick_rates, 'ban_rate': ban_rates}
win_rate_data = pd.DataFrame(data)

# remove spaces, ' and . in the championName column
win_rate_data['championName'] = win_rate_data['championName'].str.strip().str.replace("'", '').str.replace(".", '').str.replace(' ', '')

# drop duplicates and average their win rates, pick rates, and ban rates
win_rate_data = win_rate_data.groupby('championName', as_index=False).mean()

print(win_rate_data)

driver.quit()

Error getting version of chromedriver 115. Retrying with chromedriver 114 (attempt 1/5)


    championName  win_rate  pick_rate  ban_rate
0         Aatrox    0.5132     0.0945     0.219
1           Ahri    0.4983     0.1060     0.055
2          Akali    0.4874     0.0295     0.038
3         Akshan    0.5168     0.0235     0.079
4        Alistar    0.5160     0.0800     0.032
..           ...       ...        ...       ...
159         Zeri    0.4896     0.0390     0.015
160        Ziggs    0.5074     0.0115     0.001
161       Zilean    0.5074     0.0230     0.007
162          Zoe    0.5129     0.0350     0.023
163         Zyra    0.5098     0.0360     0.012

[164 rows x 4 columns]


In [10]:
win_rate_data.to_csv('win_rate_data.csv', index=False)